## Flux with PCA

### Libraries

In [ ]:
#using CUDA

using Flux              # the julia ml library
using Images            # image processing and machine vision for julia

using MLJ               # make_blobs, rmse, confmat, f1score, coerce
#using MLJFlux           # NeuralNetworkClassifier, CUDALibs
using MLDataUtils       # label, nlabel, labelfreq, stratifiedobs
using MLDatasets        # mnist

using LinearAlgebra     # pinv pseudo-inverse matrix
#using Metrics           # r2-score
using Random
using StatsBase         # standardize (normalization)
using Distributions

using Plots; gr()
using StatsPlots
using Printf

#using CSV
using DataFrames


### Functions

In [ ]:
# metrics
function printMetrics(ŷ, y)
    display(confmat(ŷ, y))
    println("accuracy: ", round(accuracy(ŷ, y); digits=3))
    println("f1-score: ", round(multiclass_f1score(ŷ, y); digits=3))
end


In [ ]:
image2Vector(M) = vec( Float32.(M) )   # 32-bits is faster on GPU

function batchImage2Vector(imagesArray3D)
    h, v, N = size(imagesArray3D)
    vectorOfImageVectors = [ image2Vector( imagesArray3D[:, :, i] ) for i in 1:N]
end

function batchImage2Matrix(imagesArray3D)
    vectorOfImageVectors = batchImage2Vector(imagesArray3D)
    M = reduce(hcat, vectorOfImageVectors)
    M'
end

function batchImage2DF(imagesArray3D)
    M = batchImage2Matrix(imagesArray3D)
    DataFrame(M, :auto)
end


### Dataset

In [ ]:
# load mnist from MLDatasets
trainX_original,      trainY_original      = MNIST.traindata()
validationX_original, validationY_original = MNIST.testdata();


In [ ]:
# split trainset, testset, validation set
Random.seed!(1)
(trainX, trainY), (testX, testY) = stratifiedobs((trainX_original, trainY_original), p = 0.7)
validationX = copy(validationX_original); validationY = copy(validationY_original)

size(trainX), size(testX), size(validationX)

### Preprocessing


In [ ]:
function preprocess(X)
    newX = batchImage2DF(X)
    #coerce!(newX)   # no need, all scitypes are Continuous in this example
    #new_y = coerce(y, OrderedFactor)
    
    return newX
end

X = preprocess(trainX);

In [ ]:
# reduce predictors
PCA = @load PCA pkg=MultivariateStats verbosity=0
reducer = PCA(pratio = 0.95)

# standardize predictors
std = Standardizer()

# execute
pipe = @pipeline reducer std
mach = MLJ.machine(pipe, X) |> fit!
X_til = MLJ.transform(mach, X);

In [ ]:
function preprocess2(X, y)
    N, d = size(X)
    Xs = X |> Matrix 
    Xs = Float32.(Xs) |> Flux.flatten
    Xs = [Xs[i,:] for i in 1:N]
    Xs = Flux.batch(Xs)
    ys = Flux.onehotbatch( Float32.(y), 0:9 )
    
    return (Xs, ys)
end

X, y = preprocess2(X_til, trainY);

### Model

In [ ]:
# model configuration
nInputs  = size(X_til)[2]
nOutputs = 10
model = Chain( Flux.Dense(nInputs, nOutputs, tanh),   # tanh is chosen as nonlinearity (Prof Mostafa lecture)
               softmax )                              # softmax scales the output to sum to one

lossFunction(X, y) = Flux.mse( model(X), y )
modelParameters    = Flux.params(model)
data               = Flux.DataLoader((X, y), batchsize=1)             # default batchsize=1
callBack           = Flux.throttle(() -> println("training"), 10);    # print every 10s


In [ ]:
numberOfEpochs = 10;

In [ ]:
# preferred for multiple epochs
epochs = 1:numberOfEpochs
@time for epoch in epochs Flux.train!(lossFunction, modelParameters, data, Flux.Descent(); cb=callBack) end

### Predict

In [ ]:
function predictOutcome(X)
    ŷ = Flux.onecold( model(X), [0:9;] )
end

ŷ = predictOutcome(X);


In [ ]:
printMetrics( coerce(ŷ, OrderedFactor), coerce(trainY, OrderedFactor) )